In [6]:
import requests
import pandas as pd
import re
import time
from datetime import datetime

In [7]:
tickerTime = 0

In [8]:
def getTickers(email):
    """
    This function retrieves all the tickers on the SEC website and returns them in a dataframe
    """
    
    headers = {'User-Agent': f"{email}"}
    startTime = time.time()
    companyTickers = requests.get(
        "https://www.sec.gov/files/company_tickers.json",
        headers=headers
        )
    endTime = time.time()
    tickerTime = endTime - startTime
    tickerDict = companyTickers.json()
    tickerDf = pd.DataFrame(index = range(len(tickerDict)), columns = ["CIK", "Ticker", "Name"])
    for i in range(len(tickerDict)): 
        tickerDf.iloc[i, 0] = str(tickerDict[str(i)]['cik_str']).zfill(10)
        tickerDf.iloc[i, 1] = tickerDict[str(i)]['ticker']
        tickerDf.iloc[i, 2] = tickerDict[str(i)]['title']
    return tickerDf

In [9]:
class Company:
    
    email = "anthonytaylor@ufl.edu"
    tickerDf = getTickers(email)

    def __init__(self, ticker):
        self.ticker = ticker
        self.cik = self.findCik()
        self.rawCompanyData = self.findRawCompanyData()
        self.incomeStatementDict = self.rawDataToIncomeStatementDict()
        self.incomeStatement = self.formIncStateFromDict()

    def findCik(self):
        cikRow = self.tickerDf[self.tickerDf['Ticker'] == self.ticker]
        cik = cikRow.iloc[0, 0]
        return cik
    
    def getCik(self):
        return self.cik 
    
    def findRawCompanyData(self):
        rawCompanyData = requests.get(
            f'https://data.sec.gov/api/xbrl/companyfacts/CIK{self.cik}.json',
            headers={'User-Agent': self.email})
        rawCompanyData = rawCompanyData.json()
        return rawCompanyData['facts']['us-gaap']
    
    def printRawCompanyDataKeys(self):
        print(self.rawCompanyData.keys())
        
    def searchRawCompanyDataKeys(self, keyword):
        matchList = []
        keyword = re.compile(keyword)
        for key in self.rawCompanyData.keys():
            if re.search(keyword, key, re.IGNORECASE):
                matchList.append(key)
        
    def rawDataToIncomeStatementDict(self):
        '''
        use regex to form condensed dictionary of just income statement items 
        '''
        incomeStatementDict = {}
        #Set revenue keywords useing re package
        revenueKeys = ["[Ss]ales[Rr]evenue[Nn]et"]
        revenueKeys = [re.compile(key) for key in revenueKeys]
        #Search each possible keyWord with each item in the data until match is found
        for keyWord in revenueKeys:
            for key in self.rawCompanyData.keys():
                if keyWord.search(key):
                    incomeStatementDict['Revenue'] = self.rawCompanyData[key]['units']['USD']
                    break
        #Do the same all the way dow nthe income statement
        costKeys = [r"[Cc]ost[Oo]f[Gs]oods[Ss]old", r'[Cc]ost[Oo]f[Rr]evenue', r'CostOfGoodsAndServicesSold']
        costKeys = [re.compile(key) for key in costKeys]
        for keyWord in costKeys:
            for key in self.rawCompanyData.keys():
                if keyWord.search(key):
                    incomeStatementDict['CostOfGoodsSold'] = self.rawCompanyData[key]['units']['USD']
                    break
        GPKeys = [r"[Gg]ross[Pp]rofit.*"]
        GPKeys = [re.compile(key) for key in GPKeys]
        for keyWord in GPKeys:
            for key in self.rawCompanyData.keys():
                if keyWord.search(key):
                    incomeStatementDict['GrossProfit'] = self.rawCompanyData[key]['units']['USD']
                    break     
        OpExKeys = [r"[Oo]perating[Ee]xpenses"]
        OpExKeys = [re.compile(key) for key in OpExKeys]
        for keyWord in OpExKeys:
            for key in self.rawCompanyData.keys():
                if keyWord.search(key):
                    incomeStatementDict['OperatingExpenses'] = self.rawCompanyData[key]['units']['USD']
                    break
        EBITKeys = [r"[Ii]ncome[Ff]rom[Oo]perations*"]
        EBITKeys = [re.compile(key) for key in EBITKeys]
        for keyWord in EBITKeys:
            for key in self.rawCompanyData.keys():
                if keyWord.search(key):
                    incomeStatementDict['EBIT'] = self.rawCompanyData[key]['units']['USD']
                    break
        preTaxIncKeys = [r"[Ii]ncome[Bb]efore[Tt]axes*"]
        PreTaxIncKeys = [re.compile(key) for key in preTaxIncKeys]
        for keyWord in PreTaxIncKeys:
            for key in self.rawCompanyData.keys():
                if keyWord.search(key):
                    incomeStatementDict['PreTaxIncome'] = self.rawCompanyData[key]['units']['USD']
                    break
        incomeKeys = [r"\b[Nn]et[Ii]ncome"]
        incomeKeys = [re.compile(key) for key in incomeKeys]
        for keyWord in incomeKeys:
            for key in self.rawCompanyData.keys():
                if keyWord.search(key):
                    incomeStatementDict['NetIncome'] = self.rawCompanyData[key]['units']['USD']
                    break
        incomeKeys = [r"[Ee]arnings[Pp]er[Ss]hare[Bb]asic"]
        incomeKeys = [re.compile(key) for key in incomeKeys]
        for keyWord in incomeKeys:
            for key in self.rawCompanyData.keys():
                if keyWord.search(key):
                    incomeStatementDict['EPS'] = self.rawCompanyData[key]['units']['USD/shares']
                    break
        return incomeStatementDict
    
    def printIncomeStatementDictKeys(self):
        print(self.incomeStatementDict.keys())
        
    def printIncomeStatementDictKeys2(self):
        print(self.incomeStatementDict['Revenue'])
        
    def formIncStateFromDict(self):
        colDict = {}
        item = 'CostOfGoodsSold'
        # iterate through dictionary to create a condensed dictionary that feeds into dataframe
        for i in range(len(self.incomeStatementDict[item])):
            #create a tuple of date times to represent a range of dates for each value 
             start = self.incomeStatementDict[item][i]['start']
             end = self.incomeStatementDict[item][i]['end']
             format = "%Y-%m-%d"
             start = datetime.strptime(start,format)
             end = datetime.strptime(end,format)
             quarter = (start.date(), end.date())
             colDict[quarter] = [self.incomeStatementDict[item][i]['val']]
        incomeDf = pd.DataFrame(colDict)
        return incomeDf

    def printIncState(self):
        pd.set_option('display.max_rows', None)
        return self.incomeStatement

In [10]:
aapl = Company("AAPL")
# if the first value of the tuple doesn't exist, delete it
aapl.printIncomeStatementDictKeys()

dict_keys(['Revenue', 'CostOfGoodsSold', 'GrossProfit', 'OperatingExpenses', 'NetIncome', 'EPS'])


In [11]:
company = Company("AAPL")
company.printIncState()

2006-10-01   2007-09-30  2008-03-30   2007-09-30  2008-09-28               \
    2007-09-29   2008-06-28  2008-06-28   2008-09-27  2008-12-27   2009-03-28   
0  16426000000  16178000000  4864000000  24294000000  7373000000  12830000000   

   2008-12-28   2008-09-28  2009-03-29   2008-09-28  ...    2022-09-25  \
   2009-03-28   2009-06-27  2009-06-27   2009-09-26  ...    2023-04-01   
0  5457000000  18581000000  5751000000  25683000000  ...  119682000000   

    2023-01-01    2022-09-25   2023-04-02    2022-09-25   2023-10-01  \
    2023-04-01    2023-07-01   2023-07-01    2023-09-30   2023-12-30   
0  52860000000  165066000000  45384000000  214137000000  64720000000   

                  2023-12-31    2023-10-01   2024-03-31  
     2024-03-30   2024-03-30    2024-06-29   2024-06-29  
0  113202000000  48482000000  159301000000  46099000000  

[1 rows x 99 columns]

In [5]:
from dataframe import MyDataFrame
df = MyDataFrame()

In [2]:
from hashmap import MyHashMap
from dataframe import MyDataFrame

import requests
import pandas as pd
import re
import time
from datetime import datetime

tickerTime = 0

def getTickers(email):
    """
    This function retrieves all the tickers on the SEC website and returns them in a dataframe
    """
    
    headers = {'User-Agent': f"{email}"}
    startTime = time.time()
    companyTickers = requests.get(
        "https://www.sec.gov/files/company_tickers.json",
        headers=headers
        )
    endTime = time.time()
    tickerTime = endTime - startTime
    tickerDict = companyTickers.json()
    tickerDf = pd.DataFrame(index = range(len(tickerDict)), columns = ["CIK", "Ticker", "Name"])
    for i in range(len(tickerDict)): 
        tickerDf.iloc[i, 0] = str(tickerDict[str(i)]['cik_str']).zfill(10)
        tickerDf.iloc[i, 1] = tickerDict[str(i)]['ticker']
        tickerDf.iloc[i, 2] = tickerDict[str(i)]['title']
    return tickerDf

class Company:
    
    email = "anthonytaylor@ufl.edu"
    tickerDf = getTickers(email)

    def __init__(self, ticker):
        self.ticker = ticker
        self.cik = self.findCik()
        self.rawCompanyData = self.findRawCompanyData()
        self.incomeStatementDict = self.rawDataToIncomeStatementDict()
        self.incomeStatement = self.formIncStateFromDict()

    def findCik(self):
        cikRow = self.tickerDf[self.tickerDf['Ticker'] == self.ticker]
        cik = cikRow.iloc[0, 0]
        return cik
    
    def getCik(self):
        return self.cik 
    
    def findRawCompanyData(self):
        rawCompanyData = requests.get(
            f'https://data.sec.gov/api/xbrl/companyfacts/CIK{self.cik}.json',
            headers={'User-Agent': self.email})
        rawCompanyData = rawCompanyData.json()
        return rawCompanyData['facts']['us-gaap']
    
    def printRawCompanyDataKeys(self):
        print(self.rawCompanyData.keys())
        
    def searchRawCompanyDataKeys(self, keyword):
        matchList = []
        keyword = re.compile(keyword)
        for key in self.rawCompanyData.keys():
            if re.search(keyword, key, re.IGNORECASE):
                matchList.append(key)
        
    def rawDataToIncomeStatementDict(self):
        '''
        use regex to form condensed dictionary of just income statement items 
        '''
        incomeStatementDict = MyDataFrame()
        #Set revenue keywords useing re package
        revenueKeys = ["[Ss]ales[Rr]evenue[Nn]et"]
        revenueKeys = [re.compile(key) for key in revenueKeys]
        #Search each possible keyWord with each item in the data until match is found
        for keyWord in revenueKeys:
            for key in self.rawCompanyData.keys():
                if keyWord.search(key):
                    incomeStatementDict['Revenue'] = self.rawCompanyData[key]['units']['USD']
                    break
        #Do the same all the way dow nthe income statement
        costKeys = [r"[Cc]ost[Oo]f[Gs]oods[Ss]old", r'[Cc]ost[Oo]f[Rr]evenue', r'CostOfGoodsAndServicesSold']
        costKeys = [re.compile(key) for key in costKeys]
        for keyWord in costKeys:
            for key in self.rawCompanyData.keys():
                if keyWord.search(key):
                    incomeStatementDict['CostOfGoodsSold'] = self.rawCompanyData[key]['units']['USD']
                    break
        GPKeys = [r"[Gg]ross[Pp]rofit.*"]
        GPKeys = [re.compile(key) for key in GPKeys]
        for keyWord in GPKeys:
            for key in self.rawCompanyData.keys():
                if keyWord.search(key):
                    incomeStatementDict['GrossProfit'] = self.rawCompanyData[key]['units']['USD']
                    break     
        OpExKeys = [r"[Oo]perating[Ee]xpenses"]
        OpExKeys = [re.compile(key) for key in OpExKeys]
        for keyWord in OpExKeys:
            for key in self.rawCompanyData.keys():
                if keyWord.search(key):
                    incomeStatementDict['OperatingExpenses'] = self.rawCompanyData[key]['units']['USD']
                    break
        EBITKeys = [r"[Ii]ncome[Ff]rom[Oo]perations*"]
        EBITKeys = [re.compile(key) for key in EBITKeys]
        for keyWord in EBITKeys:
            for key in self.rawCompanyData.keys():
                if keyWord.search(key):
                    incomeStatementDict['EBIT'] = self.rawCompanyData[key]['units']['USD']
                    break
        preTaxIncKeys = [r"[Ii]ncome[Bb]efore[Tt]axes*"]
        PreTaxIncKeys = [re.compile(key) for key in preTaxIncKeys]
        for keyWord in PreTaxIncKeys:
            for key in self.rawCompanyData.keys():
                if keyWord.search(key):
                    incomeStatementDict['PreTaxIncome'] = self.rawCompanyData[key]['units']['USD']
                    break
        incomeKeys = [r"\b[Nn]et[Ii]ncome"]
        incomeKeys = [re.compile(key) for key in incomeKeys]
        for keyWord in incomeKeys:
            for key in self.rawCompanyData.keys():
                if keyWord.search(key):
                    incomeStatementDict['NetIncome'] = self.rawCompanyData[key]['units']['USD']
                    break
        incomeKeys = [r"[Ee]arnings[Pp]er[Ss]hare[Bb]asic"]
        incomeKeys = [re.compile(key) for key in incomeKeys]
        for keyWord in incomeKeys:
            for key in self.rawCompanyData.keys():
                if keyWord.search(key):
                    incomeStatementDict['EPS'] = self.rawCompanyData[key]['units']['USD/shares']
                    break
        return incomeStatementDict
        
    def formIncStateFromDict(self):
        colDict = MyHashMap()
        item = 'CostOfGoodsSold'
        # iterate through dictionary to create a condensed dictionary that feeds into dataframe
        for i in range(len(self.incomeStatementDict[item])):
            #create a tuple of date times to represent a range of dates for each value 
             start = self.incomeStatementDict[item][i]['start']
             end = self.incomeStatementDict[item][i]['end']
             format = "%Y-%m-%d"
             start = datetime.strptime(start,format)
             end = datetime.strptime(end,format)
             quarter = (start.date(), end.date())
             colDict[quarter] = [self.incomeStatementDict[item][i]['val']]
        sorted_columns, sorted_values = colDict.to_dataframe_data()
        df = MyDataFrame(sorted_columns, sorted_values)
        return df

    def printIncState(self):
        pd.set_option('display.max_rows', None)
        print(self.incomeStatement)



# tsla = Company("TSLA")
# tsla.printRawCompanyDataKeys()
# tsla.printIncomeStatementDictKeys()
        
aapl = Company("AAPL")
# aapl.printRawCompanyDataKeys()
aapl.printIncState()
# aapl.printIncomeStatementDictKeys2()

#Keys of inital dictionary that conatins all data for one company
#Output should look like ['cik', 'entityName', 'facts']
#Facts contains all data and is a dictionary
#cik and entityName are just the name and ID of company
#print(tsla.findCompanyFacts().keys())

#the keys of the facts are ['dei', 'us-gaap'] 
#dei is irrelavlent and we need what is in us-gaap, which is a also a dictionary
#print(tsla.findCompanyFacts()['facts'].keys())

#the keys of facts are all of the line items of every financial statement that the sec keeps data on
#print(tsla.findCompanyFacts()['facts']['us-gaap'].keys())

#here I choose the first line item to look at
#every line item is a dictionary and should have keys ['label', 'description', 'units']
#label and decription only conatain a string about the line item
#units is a dictionary with one key, ['USD']
#print(tsla.findCompanyFacts()['facts']['us-gaap']['AccountsAndNotesReceivableNet'].keys())

#USD is a list
#the first element of the list is the data relating to the oldest continuously stored data point of the line item
#and each element after that should be the data relating to the line item 1 quarter later 
#print(tsla.findCompanyFacts()['facts']['us-gaap']['AccountsAndNotesReceivableNet']['units']['USD'])

MyDataFrame:
+------------------------------------------------------------+------------------------------------------------------------+------------------------------------------------------------+------------------------------------------------------------+-------------------------------------------------------------+------------------------------------------------------------+-------------------------------------------------------------+------------------------------------------------------------+------------------------------------------------------------+------------------------------------------------------------+-------------------------------------------------------------+------------------------------------------------------------+-------------------------------------------------------------+------------------------------------------------------------+------------------------------------------------------------+------------------------------------------------------------+------

In [2]:
df = MyDataFrame()
df["Age"] = [25, 30, 35]
df["City"] = ["New York", "Los Angeles", "Chicago"]

In [3]:
df

MyDataFrame:
+-------+-------------+
|   Age | City        |
+=======+=============+
|    25 | New York    |
+-------+-------------+
|    30 | Los Angeles |
+-------+-------------+
|    35 | Chicago     |
+-------+-------------+

In [7]:
from hashmap import MyHashMap
from dataframe import MyDataFrame

import requests
import pandas as pd
import re
import time
from datetime import datetime, timedelta


tickerTime = 0

def periodRangeF(colDate):
    return abs(colDate[1]-colDate[0])

def cleanDfDates(df):
    difTolerance = 3
    difTolerance = timedelta(days=difTolerance)
    one_day = timedelta(days=1)
    #Set quarter ranges
    quarterRange = 100
    quarterRange = timedelta(days=quarterRange)
    threeQuarterRange = 260
    threeQuarterRange = timedelta(days=threeQuarterRange)
    yearRange = 350
    yearRange = timedelta(days=yearRange)
    
    i = 0     
    while i < len(df.columns) - 1:
        periodRange = periodRangeF(df.columns[i])

        if (i < len(df.columns)-1):
            nextDateDiff = abs(df.columns[i+1][0]-df.columns[i][1])
        else:
            nextDateDiff = difTolerance + one_day
        
        if (periodRange > yearRange):
            if (i < 3) | (i == len(df.columns)-1):
                 df.drop(df.columns[i])
                 i=-1
            elif (abs(df.columns[i-1][1]-df.columns[i+1][0]) < nextDateDiff):
                 df.drop(df.columns[i])
                 i = i - 1
            elif nextDateDiff < difTolerance:
                if i >= 3:
                    index = df.columns.index(df.columns[i])
                    currentQuarterVal = df.data[df.columns[index]][0] - (
                        df.data[df.columns[index-1]][0] + 
                        df.data[df.columns[index-2]][0] + 
                        df.data[df.columns[index-3]][0]
                    )
                    df.data[df.columns[i]][0] = currentQuarterVal
                    currentQuarterDate = (df.columns[i-1][1] + one_day, df.columns[i][1])
                    df.rename(columns={df.columns[i]: currentQuarterDate})

        
        #column is 9 month period
        elif ((periodRange > threeQuarterRange) and (periodRange < yearRange)):
            #If the end period of the next column match up, use data to create missing quarter
            if i == 0:
                df.drop(df.columns[i])
                i=-1
            elif abs(df.columns[i+1][0]-df.columns[i-1][1]) < nextDateDiff:
                df.drop(df.columns[i])
            elif df.columns[i][1] == df.columns[i+1][1]:
                if i >= 1:
                    index = df.columns.index(df.columns[i])
                    currentQuarterVal = df.data[df.columns[index]][0] - (
                        df.data[df.columns[index-1]][0] + 
                        df.data[df.columns[index-2]][0] + 
                        df.data[df.columns[index-3]][0]
                    )
                    df.data[df.columns[i]][0] = currentQuarterVal
                    currentQuarterDate = (df.columns[i-1][1] + one_day, df.columns[i][1])
                    df.rename(columns={df.columns[i]: currentQuarterDate})
                else:
                    df.drop(df.columns[i])
                    i=-1
         
        #column is 6 month period
        elif ((periodRange > quarterRange) and (periodRange < threeQuarterRange)):
             #redudant 6 month
            if i > 0:
                prevRange = abs(df.columns[i-1][1]-df.columns[i-1][0])
                nextRange = abs(df.columns[i+1][1]-df.columns[i+1][0])
                if (prevRange < quarterRange) and (nextRange < quarterRange):
                    if (abs(df.columns[i-1][1]-df.columns[i+1][0]) < nextDateDiff):
                        df.drop(df.columns[i])
                        i = i - 1
            #If the end period of the next column match up, use data to create missing quarter
            elif df.columns[i][1] == df.columns[i+1][1]:
                index = df.columns.index(df.columns[i])
                currentQuarterVal = df.data[df.columns[index]][0] - (
                    df.data[df.columns[index-1]][0] + 
                    df.data[df.columns[index-2]][0] + 
                    df.data[df.columns[index-3]][0]
                )
                df.data[df.columns[i]][0] = currentQuarterVal
                currentQuarterDate = (df.columns[i-1][1] + one_day, df.columns[i][1])
                df.rename(columns={df.columns[i]: currentQuarterDate})
         #current column doesnt match next column sequencally 
        i += 1
    return df

def merge_my_dataframes(df1, df2):
    # Ensure both dataframes have the same columns
    if set(df1.columns) != set(df2.columns):
        raise ValueError("Both dataframes must have the same columns.")
    
    # Rename columns to avoid duplication
    df1_renamed = MyDataFrame()
    df2_renamed = MyDataFrame()
    
    for col in df1.columns:
        df1_renamed[f"{col}_1"] = df1[col]
        df2_renamed[f"{col}_2"] = df2[col]
    
    # Concatenate rows of both dataframes
    merged_df = MyDataFrame()
    for col in df1_renamed.columns:
        merged_df[col] = df1_renamed[col] + df2_renamed[col]
    
    return merged_df


def getTickers(email):
    """
    This function retrieves all the tickers on the SEC website and returns them in a dataframe
    """
    
    headers = {'User-Agent': f"{email}"}
    startTime = time.time()
    companyTickers = requests.get(
        "https://www.sec.gov/files/company_tickers.json",
        headers=headers
        )
    endTime = time.time()
    tickerTime = endTime - startTime
    tickerDict = companyTickers.json()
    tickerDf = pd.DataFrame(index = range(len(tickerDict)), columns = ["CIK", "Ticker", "Name"])
    for i in range(len(tickerDict)): 
        tickerDf.iloc[i, 0] = str(tickerDict[str(i)]['cik_str']).zfill(10)
        tickerDf.iloc[i, 1] = tickerDict[str(i)]['ticker']
        tickerDf.iloc[i, 2] = tickerDict[str(i)]['title']
    return tickerDf

def getTickers(email):
    """
    This function retrieves all the tickers on the SEC website and returns them in a dataframe
    """
    
    headers = {'User-Agent': f"{email}"}
    startTime = time.time()
    companyTickers = requests.get(
        "https://www.sec.gov/files/company_tickers.json",
        headers=headers
        )
    endTime = time.time()
    tickerTime = endTime - startTime
    tickerDict = companyTickers.json()
    tickerDf = pd.DataFrame(index = range(len(tickerDict)), columns = ["CIK", "Ticker", "Name"])
    for i in range(len(tickerDict)): 
        tickerDf.iloc[i, 0] = str(tickerDict[str(i)]['cik_str']).zfill(10)
        tickerDf.iloc[i, 1] = tickerDict[str(i)]['ticker']
        tickerDf.iloc[i, 2] = tickerDict[str(i)]['title']
    return tickerDf

class Company:
    
    email = "anthonytaylor@ufl.edu"
    tickerDf = getTickers(email)

    def __init__(self, ticker):
        self.ticker = ticker
        self.cik = self.findCik()
        self.rawCompanyData = self.findRawCompanyData()
        self.incomeStatementDict = self.rawDataToIncomeStatementDict()
        self.incomeStatement = self.formIncStateFromDict()

    def findCik(self):
        cikRow = self.tickerDf[self.tickerDf['Ticker'] == self.ticker]
        cik = cikRow.iloc[0, 0]
        return cik
    
    def getCik(self):
        return self.cik 
    
    def findRawCompanyData(self):
        rawCompanyData = requests.get(
            f'https://data.sec.gov/api/xbrl/companyfacts/CIK{self.cik}.json',
            headers={'User-Agent': self.email})
        rawCompanyData = rawCompanyData.json()
        if rawCompanyData:
            return rawCompanyData['facts']['us-gaap']
        else:
            return 0
    
    def printRawCompanyDataKeys(self):
        print(self.rawCompanyData.keys())
        
    def searchRawCompanyDataKeys(self, keyword):
        matchList = []
        keyword = re.compile(keyword)
        for key in self.rawCompanyData.keys():
            if re.search(keyword, key, re.IGNORECASE):
                matchList.append(key)
        
    def rawDataToIncomeStatementDict(self):
        '''
        use regex to form condensed dictionary of just income statement items 
        '''
        incomeStatementDict = {}
        #Set revenue keywords useing re package
        #Add back - r"[Ss]ales[Rr]evenue[Nn]et"
        revenueKeys = [r"[Rr]evenues", r"[Ss]ales[Rr]evenue[Nn]et"]
        revenueKeys = [re.compile(key) for key in revenueKeys]
        #Search each possible keyWord with each item in the data until match is found
        breakLoop = False
        for keyWord in revenueKeys:
            for key in self.rawCompanyData.keys():
                if keyWord.search(key):
                    incomeStatementDict['Revenue'] = self.rawCompanyData[key]['units']['USD']
                    breakLoop = True
                    break
            if breakLoop:
                breakLoop = False
                break
        #Do the same all the way dow nthe income statement
        costKeys = [r"[Cc]ost[Oo]f[Gs]oods[Ss]old", r'[Cc]ost[Oo]f[Rr]evenue', r'CostOfGoodsAndServicesSold']
        costKeys = [re.compile(key) for key in costKeys]
        for keyWord in costKeys:
            for key in self.rawCompanyData.keys():
                if keyWord.search(key):
                    incomeStatementDict['CostOfGoodsSold'] = self.rawCompanyData[key]['units']['USD']
                    break
            if breakLoop:
                breakLoop = False
                break
        '''
        GPKeys = [r"[Gg]ross[Pp]rofit.*"]
        GPKeys = [re.compile(key) for key in GPKeys]
        for keyWord in GPKeys:
            for key in self.rawCompanyData.keys():
                if keyWord.search(key):
                    incomeStatementDict['GrossProfit'] = self.rawCompanyData[key]['units']['USD']
                    break  
            if breakLoop:
                breakLoop = False
                break
        '''
        OpExKeys = [r"[Oo]perating[Ee]xpenses"]
        OpExKeys = [re.compile(key) for key in OpExKeys]
        for keyWord in OpExKeys:
            for key in self.rawCompanyData.keys():
                if keyWord.search(key):
                    incomeStatementDict['OperatingExpenses'] = self.rawCompanyData[key]['units']['USD']
                    break
            if breakLoop:
                breakLoop = False
                break
        # Calculatatable from GP and Op Ex
        '''
        EBITKeys = [r"[Ii]ncome[Ff]rom[Oo]perations*"]
        EBITKeys = [re.compile(key) for key in EBITKeys]
        for keyWord in EBITKeys:
            for key in self.rawCompanyData.keys():
                if keyWord.search(key):
                    incomeStatementDict['EBIT'] = self.rawCompanyData[key]['units']['USD']
                    break
            if breakLoop:
                breakLoop = False
                break    
        '''    
        #Try to get tax expense
        '''
        preTaxIncKeys = [r"[Ii]ncome[Bb]efore[Tt]axes*"]
        PreTaxIncKeys = [re.compile(key) for key in preTaxIncKeys]
        for keyWord in PreTaxIncKeys:
            for key in self.rawCompanyData.keys():
                if keyWord.search(key):
                    incomeStatementDict['PreTaxIncome'] = self.rawCompanyData[key]['units']['USD']
                    break
            if breakLoop:
                breakLoop = False
                break
         '''    
        incomeKeys = [r"\b[Nn]et[Ii]ncome[Ll]oss"]
        incomeKeys = [re.compile(key) for key in incomeKeys]
        for keyWord in incomeKeys:
            for key in self.rawCompanyData.keys():
                if keyWord.search(key):
                    incomeStatementDict['NetIncome'] = self.rawCompanyData[key]['units']['USD']
                    break
            if breakLoop:
                breakLoop = False
                break
        incomeKeys = [r"[Ee]arnings[Pp]er[Ss]hare[Bb]asic"]
        incomeKeys = [re.compile(key) for key in incomeKeys]
        for keyWord in incomeKeys:
            for key in self.rawCompanyData.keys():
                if keyWord.search(key):
                    incomeStatementDict['EPS'] = self.rawCompanyData[key]['units']['USD/shares']
                    break
            if breakLoop:
                breakLoop = False
                break
        return incomeStatementDict
    
    def printIncomeStatementDictKeys(self):
        print(self.incomeStatementDict.keys())
        
    def printIncomeStatementDictKeys2(self):
        print(self.incomeStatementDict['Revenue'])

    '''
    def formIncStateFromDict(self):
        colDict = MyHashMap()
        item = 'CostOfGoodsSold'
        # iterate through dictionary to create a condensed dictionary that feeds into dataframe
        for i in range(len(self.incomeStatementDict[item])):
            #create a tuple of date times to represent a range of dates for each value 
             start = self.incomeStatementDict[item][i]['start']
             end = self.incomeStatementDict[item][i]['end']
             format = "%Y-%m-%d"
             start = datetime.strptime(start,format)
             end = datetime.strptime(end,format)
             quarter = (start.date(), end.date())
             colDict[quarter] = [self.incomeStatementDict[item][i]['val']]
        sorted_columns, sorted_values = colDict.to_dataframe_data()
        df = MyDataFrame(sorted_columns, sorted_values)
        return df
        '''  
    
    def formIncStateFromDict(self):
        if len(self.incomeStatementDict) == 5:
            colDict = MyHashMap()
            item = ['Revenue', 'CostOfGoodsSold', 'NetIncome', 'EPS']
            # iterate through dictionary to create a condensed dictionary that feeds into dataframe
            for i in range(len(self.incomeStatementDict[item[0]])):
                #create a tuple of date times to represent a range of dates for each value 
                start = self.incomeStatementDict[item[0]][i]['start']
                end = self.incomeStatementDict[item[0]][i]['end']
                format = "%Y-%m-%d"
                start = datetime.strptime(start,format)
                end = datetime.strptime(end,format)
                quarter = (start.date(), end.date())
                colDict[quarter] = [self.incomeStatementDict[item[0]][i]['val']]
            sorted_columns, sorted_values = colDict.to_dataframe_data()
            revenueDf = MyDataFrame(sorted_columns, sorted_values)
            revenueDf = cleanDfDates(revenueDf)
            '''
            for i in range(len(self.incomeStatementDict[item[1]])):
                #create a tuple of date times to represent a range of dates for each value 
                start = self.incomeStatementDict[item[1]][i]['start']
                end = self.incomeStatementDict[item[1]][i]['end']
                format = "%Y-%m-%d"
                start = datetime.strptime(start,format)
                end = datetime.strptime(end,format)
                quarter = (start.date(), end.date())
                colDict[quarter] = [self.incomeStatementDict[item[1]][i]['val']]
            '''
            incomeDf = revenueDf
            return incomeDf
        else:
            return -1

    def printIncState(self):
        pd.set_option('display.max_rows', None)
        print(self.incomeStatement)
            
    
    
    





# tsla = Company("TSLA")
# tsla.printRawCompanyDataKeys()
# tsla.printIncomeStatementDictKeys()
        
aapl = Company("TSLA")
# aapl.printRawCompanyDataKeys()
aapl.printIncState()
# aapl.printIncomeStatementDictKeys2()

#Keys of inital dictionary that conatins all data for one company
#Output should look like ['cik', 'entityName', 'facts']
#Facts contains all data and is a dictionary
#cik and entityName are just the name and ID of company
#print(tsla.findCompanyFacts().keys())

#the keys of the facts are ['dei', 'us-gaap'] 
#dei is irrelavlent and we need what is in us-gaap, which is a also a dictionary
#print(tsla.findCompanyFacts()['facts'].keys())

#the keys of facts are all of the line items of every financial statement that the sec keeps data on
#print(tsla.findCompanyFacts()['facts']['us-gaap'].keys())

#here I choose the first line item to look at
#every line item is a dictionary and should have keys ['label', 'description', 'units']
#label and decription only conatain a string about the line item
#units is a dictionary with one key, ['USD']
#print(tsla.findCompanyFacts()['facts']['us-gaap']['AccountsAndNotesReceivableNet'].keys())

#USD is a list
#the first element of the list is the data relating to the oldest continuously stored data point of the line item
#and each element after that should be the data relating to the line item 1 quarter later 
#print(tsla.findCompanyFacts()['facts']['us-gaap']['AccountsAndNotesReceivableNet']['units']['USD'])

(datetime.date(2010, 1, 1), datetime.date(2010, 3, 31))
(datetime.date(2010, 4, 1), datetime.date(2010, 6, 30))
(datetime.date(2010, 7, 1), datetime.date(2010, 9, 30))
(datetime.date(2010, 10, 1), datetime.date(2010, 12, 31))
(datetime.date(2011, 1, 1), datetime.date(2011, 3, 31))
(datetime.date(2011, 4, 1), datetime.date(2011, 6, 30))
(datetime.date(2011, 7, 1), datetime.date(2011, 9, 30))
(datetime.date(2011, 10, 1), datetime.date(2011, 12, 31))
(datetime.date(2012, 1, 1), datetime.date(2012, 3, 31))
(datetime.date(2012, 4, 1), datetime.date(2012, 6, 30))
(datetime.date(2012, 7, 1), datetime.date(2012, 9, 30))
(datetime.date(2012, 10, 1), datetime.date(2012, 12, 31))
(datetime.date(2013, 1, 1), datetime.date(2013, 3, 31))
(datetime.date(2013, 4, 1), datetime.date(2013, 6, 30))
(datetime.date(2013, 7, 1), datetime.date(2013, 9, 30))
(datetime.date(2013, 10, 1), datetime.date(2013, 12, 31))
(datetime.date(2014, 1, 1), datetime.date(2014, 3, 31))
(datetime.date(2014, 4, 1), datetime.dat

In [12]:

import requests
import pandas as pd
import re
import time
from datetime import datetime, timedelta

tickerTime = 0

def cleanDfDates(df):
    colNames = df.columns.tolist()
    difTolerance = 5
    difTolerance = timedelta(days=difTolerance)
    one_day = timedelta(days=1)
    #check if first range is around 3 months
    quarterRange = 120
    threeQuarterRange = 260
    threeQuarterRange = timedelta(days=threeQuarterRange)
    yearRange = 350
    yearRange = timedelta(days=yearRange)
    quarterRange = timedelta(days=quarterRange)
    realRange = abs(colNames[0][1]-colNames[0][0])
    '''
    while realRange > quarterRange:
         print(colNames[0])
         print("Some") 
         del colNames[0]
         realRange = abs(colNames[0][1]-colNames[0][0])
    #Iterate through all dates to ensure one after the other format 
    '''
    i = 0     
    while i < len(colNames)-1:
         dayRange = abs(colNames[i+1][0]-colNames[i][1])
         realRange = abs(colNames[i][1]-colNames[i][0])
         #column is 6 month period
         if ((realRange > quarterRange) and (realRange < threeQuarterRange)):
             #redudant 6 month
             if i > 0:
                 prevRange = abs(colNames[i-1][1]-colNames[i-1][0])
                 nextRange = abs(colNames[i+1][1]-colNames[i+1][0])
                 if (prevRange < quarterRange) and (nextRange < quarterRange):
                     if (abs(colNames[i-1][1]-colNames[i+1][0]) < dayRange):
                         del colNames[i]
                         i = i - 1
             #If the end period of the next column match up, use data to create missing quarter
             elif colNames[i][1] == colNames[i+1][1]:
                index = df.columns.get_loc(colNames[i])
                currentQuarterVal = df.iloc[0,index] - df.iloc[0,index+1]
                df.iloc[0,i] = currentQuarterVal
                currentQuarterDate = (colNames[i][0], colNames[i+1][0] - one_day)
                df.columns = [currentQuarterDate if col == colNames[i] else col for col in df.columns]
                colNames[i] = currentQuarterDate
         #column is 9 month period
         elif ((realRange > threeQuarterRange) and (realRange < yearRange)):
             #If the end period of the next column match up, use data to create missing quarter
             if i == 0:
                 del colNames[0:i]
                 i=-1
             elif abs(colNames[i+1][0]-colNames[i-1][1]) < dayRange:
                 del colNames[i]
             elif colNames[i][1] == colNames[i+1][1]:
                if i >= 1:
                    index = df.columns.get_loc(colNames[i])
                    currentQuarterVal = df.iloc[0,index] - (df.iloc[0,index-1] + df.iloc[0,index+1])
                    df.iloc[0,i] = currentQuarterVal
                    currentQuarterDate = (colNames[i][0], colNames[i+1][0] - one_day)
                    df.columns = [currentQuarterDate if col == colNames[i] else col for col in df.columns]
                    colNames[i] = currentQuarterDate
                else:
                    del colNames[0:i]
                    i=-1
         #column is 1 year period
         elif (realRange > yearRange):
             prevRange = abs(colNames[i-1][1]-colNames[i-1][0])
             nextRange = abs(colNames[i+1][1]-colNames[i+1][0])
             if i < 3:
                 del colNames[i]
                 i=-1
             #redudant year col
             elif (abs(colNames[i-1][1]-colNames[i+1][0]) < dayRange):
                 del colNames[i]
                 i = i - 1    
             #If the end period of the next column match up, use data to create missing quarter
             elif dayRange < difTolerance:
                if i >= 3:
                    index = df.columns.get_loc(colNames[i])
                    currentQuarterVal = df.iloc[0,index] - (df.iloc[0,index-1] + df.iloc[0,index-2] + df.iloc[0,index-3])
                    df.iloc[0,i] = currentQuarterVal
                    currentQuarterDate = (colNames[i-1][1] + one_day, colNames[i][1])
                    df.columns = [currentQuarterDate if col == colNames[i] else col for col in df.columns]
                    colNames[i] = currentQuarterDate
                else:
                    del colNames[0:i]
                    i=-1
         #current column doesnt match next column sequencally 
         '''           
         elif dayRange > difTolerance:
            #check if column i+2 is continous with i, if so delete i+1, we likely have redudant info
            dayRangeNext = abs(colNames[i+2][0]-colNames[i][1])
            if dayRangeNext < difTolerance:
                print("Here to delete: ",colNames[i+1])
                del colNames[i+1]
            #if i+2 is not continous with i we likely have a 1 or 6 month year data point and we must extract the current quarter data using the prevous quarters
            else:
                #redudant data is often missing second date, this if statement filters it out to avoid error
                if len(colNames[i+1]) == 1:
                    del colNames[i+1]
                #If there is not 3 prevous entries following the 10k data we will not be able recover current quarter data
                elif i >= 3:
                    currentQuarterVal = df.iloc[0,i+1] - (df.iloc[0,i] + df.iloc[0,i-1] + df.iloc[0,i-2])
                    df.iloc[0,i+1] = currentQuarterVal
                    currentQuarterDate = (colNames[i][1] + one_day, colNames[i+1][1])
                    #colNames[i+1] = currentQuarterDate
                    #df.columns = [currentQuarterDate if j == (i+1) else name for j, name in enumerate(df.columns)]
                else:
                    del colNames[0:i+1]
                    i = 0
         '''
        #check last item to ensure 30 day range
         i = i +1 
    return df[colNames]

def getTickers(email):
    """
    This function retrieves all the tickers on the SEC website and returns them in a dataframe
    """
    
    headers = {'User-Agent': f"{email}"}
    startTime = time.time()
    companyTickers = requests.get(
        "https://www.sec.gov/files/company_tickers.json",
        headers=headers
        )
    endTime = time.time()
    tickerTime = endTime - startTime
    tickerDict = companyTickers.json()
    tickerDf = pd.DataFrame(index = range(len(tickerDict)), columns = ["CIK", "Ticker", "Name"])
    for i in range(len(tickerDict)): 
        tickerDf.iloc[i, 0] = str(tickerDict[str(i)]['cik_str']).zfill(10)
        tickerDf.iloc[i, 1] = tickerDict[str(i)]['ticker']
        tickerDf.iloc[i, 2] = tickerDict[str(i)]['title']
    return tickerDf

class Company:
    
    email = "anthonytaylor@ufl.edu"
    tickerDf = getTickers(email)

    def __init__(self, ticker):
        self.ticker = ticker
        self.cik = self.findCik()
        self.rawCompanyData = self.findRawCompanyData()
        self.incomeStatementDict = self.rawDataToIncomeStatementDict()
        self.incomeStatement = self.formIncStateFromDict()

    def findCik(self):
        cikRow = self.tickerDf[self.tickerDf['Ticker'] == self.ticker]
        cik = cikRow.iloc[0, 0]
        return cik
    
    def getCik(self):
        return self.cik 
    
    def findRawCompanyData(self):
        rawCompanyData = requests.get(
            f'https://data.sec.gov/api/xbrl/companyfacts/CIK{self.cik}.json',
            headers={'User-Agent': self.email})
        rawCompanyData = rawCompanyData.json()
        if rawCompanyData:
            return rawCompanyData['facts']['us-gaap']
        else:
            return 0
    
    def printRawCompanyDataKeys(self):
        print(self.rawCompanyData.keys())
        
    def searchRawCompanyDataKeys(self, keyword):
        matchList = []
        keyword = re.compile(keyword)
        for key in self.rawCompanyData.keys():
            if re.search(keyword, key, re.IGNORECASE):
                matchList.append(key)
        
    def rawDataToIncomeStatementDict(self):
        '''
        use regex to form condensed dictionary of just income statement items 
        '''
        incomeStatementDict = {}
        #Set revenue keywords useing re package
        #Add back - r"[Ss]ales[Rr]evenue[Nn]et"
        revenueKeys = [r"[Rr]evenues", r"[Ss]ales[Rr]evenue[Nn]et"]
        revenueKeys = [re.compile(key) for key in revenueKeys]
        #Search each possible keyWord with each item in the data until match is found
        breakLoop = False
        for keyWord in revenueKeys:
            for key in self.rawCompanyData.keys():
                if keyWord.search(key):
                    incomeStatementDict['Revenue'] = self.rawCompanyData[key]['units']['USD']
                    breakLoop = True
                    break
            if breakLoop:
                breakLoop = False
                break
        #Do the same all the way dow nthe income statement
        costKeys = [r"[Cc]ost[Oo]f[Gs]oods[Ss]old", r'[Cc]ost[Oo]f[Rr]evenue', r'CostOfGoodsAndServicesSold']
        costKeys = [re.compile(key) for key in costKeys]
        for keyWord in costKeys:
            for key in self.rawCompanyData.keys():
                if keyWord.search(key):
                    incomeStatementDict['CostOfGoodsSold'] = self.rawCompanyData[key]['units']['USD']
                    break
            if breakLoop:
                breakLoop = False
                break
        GPKeys = [r"[Gg]ross[Pp]rofit.*"]
        GPKeys = [re.compile(key) for key in GPKeys]
        for keyWord in GPKeys:
            for key in self.rawCompanyData.keys():
                if keyWord.search(key):
                    incomeStatementDict['GrossProfit'] = self.rawCompanyData[key]['units']['USD']
                    break  
            if breakLoop:
                breakLoop = False
                break
        OpExKeys = [r"[Oo]perating[Ee]xpenses"]
        OpExKeys = [re.compile(key) for key in OpExKeys]
        for keyWord in OpExKeys:
            for key in self.rawCompanyData.keys():
                if keyWord.search(key):
                    incomeStatementDict['OperatingExpenses'] = self.rawCompanyData[key]['units']['USD']
                    break
            if breakLoop:
                breakLoop = False
                break
        # Calculatatable from GP and Op Ex
        '''
        EBITKeys = [r"[Ii]ncome[Ff]rom[Oo]perations*"]
        EBITKeys = [re.compile(key) for key in EBITKeys]
        for keyWord in EBITKeys:
            for key in self.rawCompanyData.keys():
                if keyWord.search(key):
                    incomeStatementDict['EBIT'] = self.rawCompanyData[key]['units']['USD']
                    break
            if breakLoop:
                breakLoop = False
                break    
        '''    
        #Try to get tax expense
        '''
        preTaxIncKeys = [r"[Ii]ncome[Bb]efore[Tt]axes*"]
        PreTaxIncKeys = [re.compile(key) for key in preTaxIncKeys]
        for keyWord in PreTaxIncKeys:
            for key in self.rawCompanyData.keys():
                if keyWord.search(key):
                    incomeStatementDict['PreTaxIncome'] = self.rawCompanyData[key]['units']['USD']
                    break
            if breakLoop:
                breakLoop = False
                break
         '''    
        incomeKeys = [r"\b[Nn]et[Ii]ncome[Ll]oss"]
        incomeKeys = [re.compile(key) for key in incomeKeys]
        for keyWord in incomeKeys:
            for key in self.rawCompanyData.keys():
                if keyWord.search(key):
                    incomeStatementDict['NetIncome'] = self.rawCompanyData[key]['units']['USD']
                    break
            if breakLoop:
                breakLoop = False
                break
        incomeKeys = [r"[Ee]arnings[Pp]er[Ss]hare[Bb]asic"]
        incomeKeys = [re.compile(key) for key in incomeKeys]
        for keyWord in incomeKeys:
            for key in self.rawCompanyData.keys():
                if keyWord.search(key):
                    incomeStatementDict['EPS'] = self.rawCompanyData[key]['units']['USD/shares']
                    break
            if breakLoop:
                breakLoop = False
                break
        return incomeStatementDict
    
    def printIncomeStatementDictKeys(self):
        print(self.incomeStatementDict.keys())
        
    def printIncomeStatementDictKeys2(self):
        print(self.incomeStatementDict['Revenue'])
        
    def formIncStateFromDict(self):
        colDict = {}
        item = ['Revenue', 'CostOfGoodsSold', 'NetIncome', 'EPS']
        # iterate through dictionary to create a condensed dictionary that feeds into dataframe
        for i in range(len(self.incomeStatementDict[item[0]])):
            #create a tuple of date times to represent a range of dates for each value 
             start = self.incomeStatementDict[item[0]][i]['start']
             end = self.incomeStatementDict[item[0]][i]['end']
             format = "%Y-%m-%d"
             start = datetime.strptime(start,format)
             end = datetime.strptime(end,format)
             quarter = (start.date(), end.date())
             colDict[quarter] = [self.incomeStatementDict[item[0]][i]['val']]
        revenueDf = pd.DataFrame(colDict)
        revenueDf = cleanDfDates(revenueDf)
        '''
        for i in range(len(self.incomeStatementDict[item[1]])):
            #create a tuple of date times to represent a range of dates for each value 
             start = self.incomeStatementDict[item[1]][i]['start']
             end = self.incomeStatementDict[item[1]][i]['end']
             format = "%Y-%m-%d"
             start = datetime.strptime(start,format)
             end = datetime.strptime(end,format)
             quarter = (start.date(), end.date())
             colDict[quarter] = [self.incomeStatementDict[item[1]][i]['val']]
        '''
        incomeDf = revenueDf
        return incomeDf

    def printIncState(self):
        pd.set_option('display.max_columns', None)
        print(self.incomeStatement)
            
   
    
    





# tsla = Company("TSLA")
# tsla.printRawCompanyDataKeys()
# tsla.printIncomeStatementDictKeys()
pd.set_option('display.max_columns', None)
        
aapl = Company("AAPL")
aapl.printIncState()
aapl.printIncomeStatementDictKeys()
aapl.printRawCompanyDataKeys()

#Keys of inital dictionary that conatins all data for one company
#Output should look like ['cik', 'entityName', 'facts']
#Facts contains all data and is a dictionary
#cik and entityName are just the name and ID of company
#print(tsla.findCompanyFacts().keys())

#the keys of the facts are ['dei', 'us-gaap'] 
#dei is irrelavlent and we need what is in us-gaap, which is a also a dictionary
#print(tsla.findCompanyFacts()['facts'].keys())

#the keys of facts are all of the line items of every financial statement that the sec keeps data on
#print(tsla.findCompanyFacts()['facts']['us-gaap'].keys())

#here I choose the first line item to look at
#every line item is a dictionary and should have keys ['label', 'description', 'units']
#label and decription only conatain a string about the line item
#units is a dictionary with one key, ['USD']
#print(tsla.findCompanyFacts()['facts']['us-gaap']['AccountsAndNotesReceivableNet'].keys())

#USD is a list
#the first element of the list is the data relating to the oldest continuously stored data point of the line item
#and each element after that should be the data relating to the line item 1 quarter later 
#print(tsla.findCompanyFacts()['facts']['us-gaap']['AccountsAndNotesReceivableNet']['units']['USD'])

    2016-09-25   2017-01-01   2017-04-02   2017-07-02   2017-10-01  \
    2016-12-31   2017-04-01   2017-07-01   2017-09-30   2017-12-30   
0  78351000000  52896000000  45408000000  52579000000  88293000000   

    2017-12-31   2018-04-01   2018-07-01  
    2018-03-31   2018-06-30   2018-09-29  
0  61137000000  53265000000  62900000000  
dict_keys(['Revenue', 'CostOfGoodsSold', 'GrossProfit', 'OperatingExpenses', 'NetIncome', 'EPS'])
dict_keys(['AccountsPayable', 'AccountsPayableCurrent', 'AccountsReceivableNetCurrent', 'AccruedIncomeTaxesCurrent', 'AccruedIncomeTaxesNoncurrent', 'AccruedLiabilities', 'AccruedLiabilitiesCurrent', 'AccruedMarketingCostsCurrent', 'AccumulatedDepreciationDepletionAndAmortizationPropertyPlantAndEquipment', 'AccumulatedOtherComprehensiveIncomeLossAvailableForSaleSecuritiesAdjustmentNetOfTax', 'AccumulatedOtherComprehensiveIncomeLossCumulativeChangesInNetGainLossFromCashFlowHedgesEffectNetOfTax', 'AccumulatedOtherComprehensiveIncomeLossForeignCurrencyTransla